# Serial Implementation

In [45]:
%%writefile smithwaterman.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <time.h>

int blosum62mat[25][25];

int place(char a) {
    switch(a) {
        case 'A': return 0;
        case 'R': return 1;
        case 'N': return 2;
        case 'D': return 3;
        case 'C': return 4;
        case 'Q': return 5;
        case 'E': return 6;
        case 'G': return 7;
        case 'H': return 8;
        case 'I': return 9;
        case 'L': return 10;
        case 'K': return 11;
        case 'M': return 12;
        case 'F': return 13;
        case 'P': return 14;
        case 'S': return 15;
        case 'T': return 16;
        case 'W': return 17;
        case 'Y': return 18;
        case 'V': return 19;
        case 'B': return 20;
        case 'J': return 21;
        case 'Z': return 22;
        case 'X': return 23;
        default: return 24;
    }
}

int score(char a, char b) {
    int dA,dB;
    dA = place(a);
    dB = place(b);
    return blosum62mat[dA][dB];
}

void mana(char* seq1, char* seq2, int** mat) {
    int k = (int)strlen(seq1) +1 ;
    int h = (int)strlen(seq2) +1;
    int i,j;
    bool gap[k][h]; //keeps track of the gap
    for (i=0;i<k;i++) {
        for(j=0;j<h;j++) {
            if(i==0||j==0) {
                mat[i][j]=0;
                gap[i][j]=false;
            }
            else {
                int hgapsc = (!gap[i-1][j]) ? mat[i-1][j]-5 : mat[i-1][j]-1;
                int vgapsc = (!gap[i][j-1]) ? mat[i][j-1]-5 : mat[i][j-1]-1;
                int xscore = mat[i-1][j-1]+score(seq1[i-1],seq2[j-1]);
                if (hgapsc > vgapsc && hgapsc > xscore) {
                    mat[i][j]=hgapsc;
                    gap[i][j]=true;
                } else if (vgapsc > hgapsc && vgapsc > xscore) {
                    mat[i][j]=vgapsc;
                    gap[i][j]=true;
                } else {
                    mat[i][j] = xscore;
                    gap[i][j] = false;
                }
                if (mat[i][j]<0) {
                    mat[i][j]=0;
                    gap[i][j] = true;
                }
            }
        }
    }
}

void traceback(char* seq1, char* seq2, int** arr ){
    int j=0, k= 0;
    int j_max = 0,  k_max = 0, max_val = 0;

    for(j = 0; j < strlen(seq1) + 1; j++){
        for(k = 0; k < strlen(seq2) + 1; k++){
            if(arr[j][k] > max_val){
                max_val = arr[j][k];
                j_max = j;
                k_max = k;
            }
        }
    }
    j = j_max;
    k = k_max;
    printf("\nIndex start at %d %d \n", j, k);
    char fin_seq1 [j_max + k_max + 1];
    char fin_seq2 [k_max + j_max + 1];
    int l =0;

    while (arr[j][k] !=0 && j>0 && k>0) {
        int score_diagonal = arr[j-1][k-1];
        int score_left = arr[j][k-1];
        int score_up = arr[j-1][k];

        if(score_diagonal >= score_left && score_diagonal >= score_up) {
            j--;
            k--;
            fin_seq1[l] = seq1[j];
            fin_seq2[l] = seq2[k];
        }
        else if(score_left >= score_diagonal && score_left >= score_up){
            k--;
            fin_seq1[l] = '-';
            fin_seq2[l] = seq2[k];
        }else{
            j--;
            fin_seq1[l] = seq1[j];
            fin_seq2[l] = '-';
        }
        l++;
    }

    fin_seq1[l] = '\0';
    fin_seq2[l] = '\0';

    for(int i = strlen(fin_seq1) -1; i >= 0 ; i--){
        printf("%c", fin_seq1[i]);

    }
    printf("\n");
    for(int i = strlen(fin_seq1) -1; i >= 0 ; i--){
        printf("%c", fin_seq2[i]);
    }
    printf("\n");
}

char rand_prot(int x) {
    switch(x) {
        case 0: return 'A';
        case 1: return 'R';
        case 2: return 'N';
        case 3: return 'D';
        case 4: return 'C';
        case 5: return 'Q';
        case 6: return 'E';
        case 7: return 'G';
        case 8: return 'H';
        case 9: return 'I';
        case 10: return 'L';
        case 11: return 'K';
        case 12: return 'M';
        case 13: return 'F';
        case 14: return 'P';
        case 15: return 'S';
        case 16: return 'T';
        case 17: return 'W';
        case 18: return 'Y';
        case 19: return 'V';
        case 20: return 'B';
        case 21: return 'J';
        case 22: return 'Z';
        case 23: return 'X';
        default: return '0';
    }

}



int main() {

    FILE* fp;
    int i=0,j=0;

    char c;
    size_t seq1_size = 1 << 7; //7, 8, 9
    size_t seq2_size = 1 << 10; //10, 11, 12, 13
    char *exseq1, *exseq2;
    int** mat;

    mat = (int**)malloc((seq1_size + 1) * sizeof(int*));
    for (int i = 0; i < seq1_size +1 ; i++)
        mat[i] = (int*)malloc((seq2_size +1) * sizeof(int));

    exseq1 = (char*)malloc((seq1_size + 1) * sizeof(char));
    exseq2 = (char*)malloc((seq2_size + 1) * sizeof(char));

    srand(time(0));
    int temp;
    for (i = 0; i < seq1_size; i++) {
        temp = rand() % 24;
        exseq1[i] = rand_prot(temp);
    }

    for (i = 0; i < seq2_size; i++) {
        temp = rand() % 24;
        exseq2[i] = rand_prot(temp);
    }

    int loope = 10;

    exseq1[seq1_size] = '\0';
    exseq2[seq2_size] = '\0';

    if((fp = fopen("BLOSUM62.txt", "r"))==NULL) return 1;
    fscanf(fp, "%*[^\n]\n");
    fscanf(fp, "%*[^\n]\n");

    for(i = 0; i<25; i++) {
        fscanf(fp, "%c", &c);
        for(j = 0; j<25; j++) {
            fscanf(fp, "%d", &blosum62mat[i][j]);
        }
        fscanf(fp, "%c", &c);
    }

    clock_t startloop, endloop;


    mana(exseq1, exseq2, mat);

    startloop = clock();
    for (i = 0;i < loope; i++) {
        mana(exseq1,exseq2, mat);
    }
    endloop = clock();
    double timetaken = (endloop-startloop)*1e3/CLOCKS_PER_SEC;

    traceback(exseq1, exseq2, mat);
    printf("total kernel time(ms): %lf\naverage kernel time(ms):%lf", timetaken, timetaken/10);

    fclose(fp);
    free(mat);
    free(exseq1);
    free(exseq2);
    return 0;
}

Overwriting smithwaterman.c


In [46]:
%%shell
gcc smithwaterman.c -o smithwaterman

In [47]:
%%shell
./smithwaterman


Index start at 123 351 
WGLPTMLCRPMNZEFKKWPHMG--LZSHJBQ-HH---------------------PTKJHSGCWPWVCXHSNWQNQQMC--BJWIYRWHIBQ-W---MQ--PQ--KGA--SZIZ-T-----YC-YARZBFKFZQPH-EACDI-TASFDPPYM-SWBIFEIEQJVTLYP
W--PTMI----NSY-GFWSBLASGFESNZSHQHHRDYGMIXALRTCRQZHYAIIGPTPH-EJ-W---------DJR-C-AVDFBVFGB-JDFDWPJWSZTQPZPFVGTHAFRVTETKKSAIYQCMVRQNSZHEBPVH--FDHFTIHISTMVGMCN-LHEJRKAT-RHP
total kernel time(ms): 61.216000
average kernel time(ms):6.121600



---



# CUDA Implementation

In [84]:
%%writefile smithwaterman.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <time.h>

__constant__ int blosum62mat[25][25];

__device__ int place(char a) {
    switch(a) {
        case 'A': return 0;
        case 'R': return 1;
        case 'N': return 2;
        case 'D': return 3;
        case 'C': return 4;
        case 'Q': return 5;
        case 'E': return 6;
        case 'G': return 7;
        case 'H': return 8;
        case 'I': return 9;
        case 'L': return 10;
        case 'K': return 11;
        case 'M': return 12;
        case 'F': return 13;
        case 'P': return 14;
        case 'S': return 15;
        case 'T': return 16;
        case 'W': return 17;
        case 'Y': return 18;
        case 'V': return 19;
        case 'B': return 20;
        case 'J': return 21;
        case 'Z': return 22;
        case 'X': return 23;
        default: return 24;
    }
}

__device__ int score(char a, char b) {
    int dA,dB;
    dA = place(a);
    dB = place(b);
    return blosum62mat[dA][dB];
}

__global__ void mana(char* seq1, const int row, char* seq2, const int col, int* scoringmat, bool* gap, int diag, size_t n) {
    int j = threadIdx.x + blockIdx.x*blockDim.x;
    int stride = blockDim.x * gridDim.x;
    for(int k = j; k < n; k+=stride) {
      int i = diag - j;
      //printf("thread: %d, block: %d, j:%d, i:%d\n",threadIdx.x, blockIdx.x, j, i);
      if (i >= 0 && i < row  && j < col) {
        if (i == 0 || j == 0) {
            scoringmat[i * col + j] = 0;
            gap[i * col + j] = false;
        } else {
        int hgapsc, vgapsc, xscore=1;
        if (!gap[(i-1)*col+j]) {
        hgapsc = scoringmat[(i-1)*col + j] - 5;
        } else {
        hgapsc = scoringmat[(i-1)*col+j] - 1;
        }
        if (!gap[i*col+(j-1)]) {
          vgapsc = scoringmat[i*col+(j-1)] - 5;
        } else {
          vgapsc = scoringmat[i*col+(j-1)] - 1;
        }
        xscore = scoringmat[(i-1) * col + (j-1)] + score(seq1[i-1],seq2[j-1]);
        if (vgapsc > hgapsc && vgapsc > xscore) {
            scoringmat[i * col + j] = vgapsc;
            gap[i * col + j] = true;
        } else if (hgapsc > vgapsc && hgapsc > xscore) {
            scoringmat[i * col + j] = hgapsc;
            gap[i * col + j] = true;
        } else {
            scoringmat[i * col + j] = xscore;
            gap[i * col + j] = false;
        }
        if (scoringmat[i * col + j] < 0) {
            scoringmat[i * col + j] = 0;
            gap[i * col + j] = true;
        }
      }
    }
  }
}

void traceback(char* seq1, char* seq2, int* scoringmat, const int col, const int row ){
    int j=0, k= 0;
    int j_max = 0,  k_max = 0, max_val = 0;

    for(j = 0; j < row; j++){
        for(k= 0; k < col; k++){
            if(scoringmat[j*col+k] > max_val){
                max_val = scoringmat[j*col+k];
                j_max = j;
                k_max = k;
            }
        }
    }
    j = j_max;
    k = k_max;
    printf("\nIndex start at %d %d \n", j, k);
    char fin_seq1 [j_max + k_max + 1];
    char fin_seq2 [k_max + j_max + 1];
    int l =0;

     while (scoringmat[j*col+k] !=0 && j>0 && k>0) {
        int score_diagonal = scoringmat[(j-1)*col+(k-1)];
        int score_left = scoringmat[j*col+(k-1)];
        int score_up = scoringmat[(j-1)*col+k];

        if(score_diagonal >= score_left && score_diagonal >= score_up) {
            j--;
            k--;
            fin_seq1[l] = seq1[j];
            fin_seq2[l] = seq2[k];
        }
        else if(score_left >= score_diagonal && score_left >= score_up){
            k--;
            fin_seq1[l] = '-';
            fin_seq2[l] = seq2[k];
        }else{
            j--;
            fin_seq1[l] = seq1[j];
            fin_seq2[l] = '-';
        }
        l++;
    }

    fin_seq1[l] = '\0';
    fin_seq2[l] = '\0';

    for(int i = strlen(fin_seq1) -1; i >= 0 ; i--){
        printf("%c", fin_seq1[i]);

    }
    printf("\n");
    for(int i = strlen(fin_seq1) -1; i >= 0 ; i--){
        printf("%c", fin_seq2[i]);
    }
    printf("\n");
}

char rand_prot(int x) {
    switch(x) {
        case 0: return 'A';
        case 1: return 'R';
        case 2: return 'N';
        case 3: return 'D';
        case 4: return 'C';
        case 5: return 'Q';
        case 6: return 'E';
        case 7: return 'G';
        case 8: return 'H';
        case 9: return 'I';
        case 10: return 'L';
        case 11: return 'K';
        case 12: return 'M';
        case 13: return 'F';
        case 14: return 'P';
        case 15: return 'S';
        case 16: return 'T';
        case 17: return 'W';
        case 18: return 'Y';
        case 19: return 'V';
        case 20: return 'B';
        case 21: return 'J';
        case 22: return 'Z';
        case 23: return 'X';
        default: return '0';
    }
}

int main() {
    FILE* fp;
    int i = 0, j = 0;
    char c;
    int h_blosum62mat[25][25];

    size_t seq1_size = 1 << 9; //7, 8, 9
    size_t seq2_size = 1 << 13; //10, 11, 12, 13

    char exseq1[seq1_size + 1];
    char exseq2[seq2_size + 1];

    srand(time(0));
    int temp;
    for (i = 0; i < seq1_size; i++) {
        temp = rand() % 24;
        exseq1[i] = rand_prot(temp);
    }

    for (i = 0; i < seq2_size; i++) {
        temp = rand() % 24;
        exseq2[i] = rand_prot(temp);
    }

    int loope = 10;

    exseq1[seq1_size] = '\0';
    exseq2[seq2_size] = '\0';



    if ((fp = fopen("BLOSUM62.txt", "r")) == NULL) return 1;

    fscanf(fp, "%*[^\n]\n");
    fscanf(fp, "%*[^\n]\n");

    for (i = 0; i < 25; i++) {
        fscanf(fp, "%c", &c);
        for (j = 0; j < 25; j++) {
            fscanf(fp, "%d", &h_blosum62mat[i][j]);
        }
        fscanf(fp, "%c", &c);
    }

    fclose(fp);

    cudaMemcpyToSymbol(blosum62mat, h_blosum62mat, sizeof(int) * 25 * 25);

    const size_t row = (size_t)strlen(exseq1) + 1;
    const size_t col = (size_t)strlen(exseq2) + 1;
    const size_t ARRAY_BYTES = row * col * sizeof(int);
    const size_t GAP_ARRAY_BYTES = row * col * sizeof(bool);

    // declare array
    int* scoringmat;
    bool* gapmat;
    cudaMallocManaged(&scoringmat, ARRAY_BYTES);
    cudaMallocManaged(&gapmat, GAP_ARRAY_BYTES);
    char* d_seq1;
    char* d_seq2;
    cudaMallocManaged(&d_seq1, row * sizeof(char));
    cudaMallocManaged(&d_seq2, col * sizeof(char));


    // get gpu id
    int device = -1;
    cudaGetDevice(&device);
    cudaMemPrefetchAsync(d_seq1, row * sizeof(char), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_seq2, col * sizeof(char), cudaCpuDeviceId, NULL);

    cudaMemcpy(d_seq1, exseq1, row * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_seq2, exseq2, col * sizeof(char), cudaMemcpyHostToDevice);

    cudaMemcpy(d_seq1, exseq1, row * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_seq2, exseq2, col * sizeof(char), cudaMemcpyHostToDevice);


    cudaMemAdvise(scoringmat, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(gapmat, GAP_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, device);
    cudaMemPrefetchAsync(scoringmat, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(gapmat, GAP_ARRAY_BYTES, device, NULL);

    int diag_count = row + col - 1;

    int numThreads = 256;
    int max = (row<col) ? col : row;

    for (i = 0;i < loope; i++) {
        for (int i = 0; i < diag_count; i++) {
            mana<<<1, numThreads>>>(d_seq1, row, d_seq2, col, scoringmat, gapmat, i, ARRAY_BYTES/4/max);
        }
    }

    cudaDeviceSynchronize();
    cudaMemPrefetchAsync(scoringmat, ARRAY_BYTES, cudaCpuDeviceId, NULL);

    traceback(exseq1, exseq2, scoringmat, col, row);

    cudaFree(scoringmat);
    cudaFree(gapmat);
    cudaFree(d_seq1);
    cudaFree(d_seq2);

    return 0;
}

Overwriting smithwaterman.cu


In [85]:
%%shell
nvcc smithwaterman.cu -o smithwaterman

In [86]:
%%shell
nvprof ./smithwaterman

==7767== NVPROF is profiling process 7767, command: ./smithwaterman

Index start at 278 250 
EHYNTIMTPRDKAAHJWYVMN--CEWHJCZSKJFKSGDTKMGFSLQSYYAFQQGRKRLHJASAZCKFDGESVQBNS-WWI-KLPWSPYNSBLCJYQV--ZHHY-SEINKJCN----PPCRALLGAJ-IGWKM-PQWRBWNPRHSDMMHE-YPLKPJZJFVZ-JCRIVXYBVDPRC---XH-HDBHIEZBWCIGRPNSKG-TMYGEENN-JVEM-DP-SSPTJMMSTFZICW-TGTRB--LH---YHD-BJZ-D--IY-ITIAIGCFPFVC----P-------SJGZFHLSHQTKW-FN--MXBZGQP-TCYZ-------YXW
EF-BTAT-MQDG--EPWJIFDFJXRFH--BK-XN--HXTBQETJ-ZA---IDD-----H----NYRSDG--YHBLLRYZFIRFIWRPS--WC-FIQIFPDXGYYPQDDSQCRVILBPKE-SYFAZDL-DCSIGPW-R----H-XE----KY------F-HXEC---AW-TSBPZPCXCXIH-D-----AWNENDPNNYBG--YKPZCXGVVJWLRHPAWPVIAJS-----YWZBVHINNMHRTGH-KD-JJQXQKSSYDTLAAC-WZRC-ENXAPGRGGRLLSXGPJYJCMT-BZWB-DNLB-XHP-PICFNDBBEEKQFW-
==7767== Profiling application: ./smithwaterman
==7767== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.87%  267.61ms     87050  3.0740us  1.1840us  29.792us  mana(char*, int, char*, int